In [ ]:
# layer1/notebooks/02_validation_testing.ipynb
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Validation Testing\n",
    "\n",
    "Test the validation module with various data quality issues."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.insert(0, '..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "from layer1.core.validation import DataValidator, ClaimSchema\n",
    "from pydantic import ValidationError"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Test Individual Validation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Valid claim\n",
    "valid_claim = {\n",
    "    'months_as_customer': 24,\n",
    "    'age': 35,\n",
    "    'policy_annual_premium': 1200.0,\n",
    "    'incident_severity': 'Minor Damage',\n",
    "    'total_claim_amount': 5000.0,\n",
    "    'injury_claim': 2000.0,\n",
    "    'property_claim': 2000.0,\n",
    "    'vehicle_claim': 1000.0,\n",
    "    'incident_type': 'Single Vehicle',\n",
    "    'collision_type': 'Front Collision',\n",
    "    'authorities_contacted': 'Police',\n",
    "    'witness_present': 'Yes',\n",
    "    'police_report_available': 'Yes'\n",
    "}\n",
    "\n",
    "try:\n",
    "    validated = ClaimSchema(**valid_claim)\n",
    "    print(\"✓ Valid claim accepted\")\n",
    "except ValidationError as e:\n",
    "    print(f\"✗ Error: {e}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Invalid claim (age too high)\n",
    "invalid_claim = valid_claim.copy()\n",
    "invalid_claim['age'] = 150\n",
    "\n",
    "try:\n",
    "    validated = ClaimSchema(**invalid_claim)\n",
    "    print(\"✗ Should have rejected invalid claim\")\n",
    "except ValidationError as e:\n",
    "    print(f\"✓ Correctly rejected: {e.errors()[0]['msg']}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Test Batch Validation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create test data with issues\n",
    "test_data = pd.DataFrame({\n",
    "    'months_as_customer': [24, -5, 36, 12],\n",
    "    'age': [35, 45, 150, 28],  # 150 is invalid\n",
    "    'policy_annual_premium': [1200.0, 1500.0, 1000.0, -100.0],  # negative invalid\n",
    "    'incident_severity': ['Minor Damage', 'Invalid', 'Major Damage', 'Trivial Damage'],\n",
    "    'total_claim_amount': [5000.0, 15000.0, 800.0, 1000.0],\n",
    "    'injury_claim': [2000.0, 5000.0, 0.0, 400.0],\n",
    "    'property_claim': [2000.0, 5000.0, 500.0, 400.0],\n",
    "    'vehicle_claim': [1000.0, 5000.0, 300.0, 200.0],\n",
    "    'incident_type': ['Single Vehicle', 'Multi-Vehicle', 'Parked Car', 'Vehicle Theft'],\n",
    "    'collision_type': ['Front Collision', 'Rear Collision', 'No Collision', 'Side Collision'],\n",
    "    'authorities_contacted': ['Police', 'Police', 'None', 'Fire'],\n",
    "    'witness_present': ['Yes', 'No', 'No', 'Yes'],\n",
    "    'police_report_available': ['Yes', 'Yes', 'No', 'Unknown']\n",
    "})\n",
    "\n",
    "validator = DataValidator()\n",
    "valid_df, report = validator.validate_batch(test_data)\n",
    "\n",
    "print(f\"Valid records: {len(valid_df)}/{len(test_data)}\")\n",
    "print(f\"\\nError breakdown:\")\n",
    "for error, count in report['error_breakdown'].items():\n",
    "    print(f\"  - {error}: {count}\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}